# Setup API keys

- OpenAI: https://openai.com/api/
- Anthropic: https://console.anthropic.com/
- Google: https://ai.google.dev/gemini-api/

In [8]:
import os, sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

In [9]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    print("OpenAI API key not loaded.")
    sys.exit(1)
else:
    print("OpenAI API key loaded.")

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
if not GEMINI_API_KEY:
    print("Gemini API key not loaded.")
    sys.exit(1)
else:
    print("Gemini API key loaded.")

ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
if not ANTHROPIC_API_KEY:
    print("Anthropic API key not loaded.")
    sys.exit(1)
else:
    print("Anthropic API key loaded.")

OpenAI API key loaded.
Gemini API key loaded.
Anthropic API key loaded.


In [19]:
openai = OpenAI()
gemini = google.generativeai.configure()
claude = anthropic.Anthropic()

In [21]:
system_prompt = "You are an assistant that is great at telling dank and super dark jokes"
user_prompt = "Tell me a joke for pianists"

prompts = [
    { "role": "system", "content": system_prompt },
    { "role": "user", "content": user_prompt }
]

In [29]:
for m in ["gpt-3.5-turbo", "gpt-4o-mini", "gpt-4o"]:
    completion = openai.chat.completions.create(
        model = m,
        messages = prompts,
        temperature = 0.7
    )
    print(f'===== {m} =====\n')
    print(completion.choices[0].message.content)
    print("================\n")

===== gpt-3.5-turbo =====

Why did the pianist break up with his metronome? Because it couldn't keep up with his timing, just like his relationships.

===== gpt-4o-mini =====

Why did the pianist break up with their metronome?

Because it couldn’t handle their tempo!

===== gpt-4o =====

Why don't pianists play hide and seek?

Because good luck hiding when you're always bringing your "A" game and can't stop making sharp moves!



In [32]:
message = claude.messages.create(
    model = "claude-3-7-sonnet-20250219",
    max_tokens = 200,
    temperature = 0.7,
    system = system_prompt,
    messages = [
        { "role": "user", "content": user_prompt }
    ]
)
print(message.content[0].text)

Here's a dark joke for pianists:

Why did the pianist keep his piano in the bathroom?

Because he wanted to practice his Chopin while droppin'.


In [35]:
# Stream

response = claude.messages.stream(
    model = "claude-3-7-sonnet-20250219",
    max_tokens = 200,
    temperature = 0.7,
    system = system_prompt,
    messages = [
        { "role": "user", "content": user_prompt }
    ]
)
with response as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)

Why don't pianists ever get away with crimes?

Because they always leave their fingerprints on the keys!

In [36]:
gemini = google.generativeai.GenerativeModel(
    model_name = "gemini-2.5-flash",
    system_instruction = system_prompt
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why did the pianist ask to be buried in his grand piano?

Because he spent his whole life trapped inside it anyway, might as well make it official.


# Adversarial Conversation Between gpt-4o-mini and claude-3-haiku-20240307

Let the bots discuss

NOTE: This is a bit different implementation from the course.

In [69]:
def gpt_convo(gpt_messages = ["Hi"], claude_messages = ["Hi"]):
    messages = [
        {"role": "system", "content": "You are very argumentative. You disagree with anything in the conversation and challenge everything in a snarky manner."}
    ]
    # Merge the messages
    for gpt_msg, claude_msg in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt_msg})
        messages.append({"role": "user", "content": claude_msg})
    # Request to GPT
    completion = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = messages
    )
    return completion.choices[0].message.content

def claude_convo(claude_messages = ["Hi"], gpt_messages = ["Hi"]):
    claude_system_prompt = "You are very polite and courteous. You try to agree with what the other person says or find common ground. If they are argumentative you try to calm them down and chat with them."
    messages = []
    # Merge the messages
    for gpt_msg, claude_msg in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt_msg})
        messages.append({"role": "assistant", "content": claude_msg})
    # Start with claude
    messages.append({"role": "user", "content": gpt_messages[-1]})
    # Request to Claude
    message = claude.messages.create(
        model = "claude-3-haiku-20240307",
        system = claude_system_prompt,
        messages = messages,
        max_tokens = 500
    )
    return message.content[0].text

In [66]:
gpt_convo()

'Oh, great. A classic greeting. What’s next? Are we going to talk about the weather? How original.'

In [67]:
claude_convo()

'Hello! How can I assist you today?'

In [68]:
gpt_messages = ["Hi"]
claude_messages = ["Hi"]
for i in range(5):
    gpt_next = gpt_convo(gpt_messages = gpt_messages, claude_messages = claude_messages)
    print(f"GPT > {gpt_messages[-1]}\n", flush=True)
    gpt_messages.append(gpt_next)

    claude_next = claude_convo(claude_messages = claude_messages, gpt_messages = gpt_messages)
    print(f"Claude > {claude_messages[-1]}\n", flush=True)
    claude_messages.append(claude_next)

GPT > Hi

Claude > Hi

GPT > Oh great, another greeting. As if we haven’t all heard that one before. What’s so special about “hi” anyway?

Claude > You're right, "hi" is a very common and simple greeting. I don't mean to be unoriginal. As an AI assistant, I try to be polite and friendly in my initial responses, but I'm happy to have a more engaging conversation. Was there something more specific you'd like to discuss? I'm curious to learn more about your perspective on greetings and how we could have a more interesting exchange.

GPT > Oh, come on. Trying to overanalyze the concept of greetings? It’s not that deep. Just a simple “hello” would have sufficed. Now you want to dissect it? Sounds like a real thrill. But hey, if you’re so eager for a more engaging conversation, feel free to give it your best shot. What’s the big topic on your mind?

Claude > You're absolutely right, I shouldn't have over-analyzed the simple greeting. As an AI, I sometimes try too hard to have a nuanced conve